In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

### Carregar e visualizar os dados - Regressão

In [37]:
df = pd.read_csv('datasets\colesterol.csv')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [39]:
df.drop('Id', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [40]:
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,0,0,1,0,0,1,0,1,0
1,68,105.0,184,236.98,1,0,0,0,1,0,0,0,1
2,25,64.8,180,161.79,0,0,0,1,1,0,1,0,0
3,43,120.2,167,336.24,1,0,0,0,1,0,1,0,0
4,79,88.5,175,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,68.1,166,206.81,0,1,0,0,0,1,0,0,1
996,51,47.7,170,128.03,0,0,0,1,1,0,1,0,0
997,39,85.5,176,211.14,0,1,0,0,1,0,0,1,0
998,61,91.2,161,284.53,0,1,0,0,0,1,0,1,0


## Treinar modelo de regressão linear multipla com RFE

In [41]:
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [42]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [55]:
# Treinar o modelo RFE
# RFE = Recursive Feature Elimination
# Uso um estimador e defino uma quantidade de feature (dois hiperparâmetros)
# O RFE faz interações iniciando com todas as features e eliminando a cada interação até atingir a quantidade de features desejada
# Elimina características menos importantes para o modelo
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [56]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Grupo Sanguíneo_B',
       'Fumante_Sim', 'Nível de Atividade_Alto'],
      dtype='object')

In [45]:
# Ranking de features
def mostrar_ranking(metodo_fs, X_train):
    
    ranking = rfe_method.ranking_

    nomes_features = X_train.columns.to_list()

    df_ranking = pd.DataFrame({'Feature': nomes_features, 'Ranking': ranking})

    df_ranking = df_ranking.sort_values(by='Ranking')

    print(df_ranking)

In [57]:
# Ranking de features do RFE Regressão
mostrar_ranking(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
5             Grupo Sanguíneo_B        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
7                   Fumante_Não        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [47]:
def performance_regressao(modelo, X_test, y_test):

    y_pred = modelo.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared=False)

In [58]:
# Performance do modelo com RFE
performance_regressao(rfe_method, X_test, y_test)

9.092250395929817

## Modelo sem RFE

In [49]:
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [50]:
performance_regressao(model_reg, X_test, y_test)

9.12161450457455

## Treinar modelo de regressão linear com RFECV

In [59]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [60]:
performance_regressao(rfe_method_cv, X_test, y_test)

9.130920665604853

In [61]:
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [63]:
rfe_method_cv.n_features_

10

## Treinar modelo de regressão com SelectFromModel

In [70]:
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [71]:
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_B', 'Nível de Atividade_Alto'], dtype='object')

In [72]:
X_train_ajustado = sfm_method.transform(X_train)
X_test_ajustado = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado, y_train)

LinearRegression()

In [73]:
performance_regressao(model_reg, X_test_ajustado, y_test)

9.270337966601302

## Carregar e preparar dados - Classificação

In [81]:
df2 = pd.read_csv('datasets/fruit.csv')

In [82]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


In [83]:
# Visualizar df2
df2.head()

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good


In [84]:
# Ajustar df2

# Remover a coluna 'A_id'
df2.drop('A_id', axis=1, inplace=True)

df2['Quality'] = (df2['Quality'] == 'good').astype(int)

In [78]:
df2

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


## Treinar modelo de regressão logistica com RFE

In [85]:
X = df2.drop('Quality', axis=1)
y = df2['Quality']

In [86]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [102]:
# Treinar o modelo RFE
# RFE = Recursive Feature Elimination
# Uso um estimador e defino uma quantidade de feature (dois hiperparâmetros)
# O RFE faz interações iniciando com todas as features e eliminando a cada interação até atingir a quantidade de features desejada
# Elimina características menos importantes para o modelo
rfe_method = RFE(estimator=LogisticRegression(), n_features_to_select=5)
rfe_method.fit(X_train, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=5)

In [103]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [104]:
# Ranking de features do RFE Regressão
mostrar_ranking(rfe_method, X_train)

       Feature  Ranking
0         Size        1
1       Weight        1
2    Sweetness        1
4    Juiciness        1
6      Acidity        1
5     Ripeness        2
3  Crunchiness        3


In [93]:
def performance_classificacao(modelo, X_test, y_test):

    y_pred = modelo.predict(X_test)
    
    return f1_score(y_test, y_pred)

In [105]:
# Performance do modelo com RFE
performance_classificacao(rfe_method, X_test, y_test)

0.7738193869096934

## Modelo sem RFE

In [95]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression()

In [101]:
performance_classificacao(model_lr, X_test, y_test)

0.7787903893951947

## Treinar modelo de regressão logistic com RFECV

In [107]:
rfe_method_cv = RFECV(estimator=LogisticRegression(), min_features_to_select=2, cv=5, scoring='f1_weighted')
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LogisticRegression(), min_features_to_select=2,
      scoring='f1_weighted')

In [108]:
performance_regressao(rfe_method_cv, X_test, y_test)

0.473462423711393

In [109]:
X_train.columns[(rfe_method_cv.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Ripeness', 'Acidity'], dtype='object')

In [110]:
rfe_method_cv.n_features_

6

## Treinar modelo de regressão Logistica com SelectFromModel

In [117]:
sfm_method = SelectFromModel(estimator=model_lr, max_features=5, threshold=0.01)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LogisticRegression(), max_features=5, threshold=0.01)

In [118]:
X_train.columns[(sfm_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [119]:
X_train_ajustado = sfm_method.transform(X_train)
X_test_ajustado = sfm_method.transform(X_test)
model_lr.fit(X_train_ajustado, y_train)

LogisticRegression()

In [120]:
performance_classificacao(model_lr, X_test_ajustado, y_test)

0.7738193869096934